In [4]:
%%bash

# Instal Java
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [5]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# **01 - Leitura de dados**

In [6]:
url = "https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv"

# Desenvolva seu código de leitura aqui:
from pyspark import SparkFiles
from pyspark.sql.functions import *
spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("titanic.csv"), header=True, inferSchema= True, sep=';')

# **02 - Algumas análises preliminares**

In [7]:
# 1) O schema da tabela
# Desenvolva sua resposta aqui:

df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
# 2) Os 10 primeiros casos
# Desenvolva sua resposta aqui:

df.limit(10).show(truncate = False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

In [9]:
# 3) Apenas as pessoas que sobreviveram
# Desenvolva sua resposta aqui:

df.where("Survived == 1")\
  .select('Name')\
  .show(df.count(),truncate = False)

+----------------------------------------------------------------------------------+
|Name                                                                              |
+----------------------------------------------------------------------------------+
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)                               |
|Heikkinen, Miss. Laina                                                            |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)                                      |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)                                 |
|Nasser, Mrs. Nicholas (Adele Achem)                                               |
|Sandstrom, Miss. Marguerite Rut                                                   |
|Bonnell, Miss. Elizabeth                                                          |
|Hewlett, Mrs. (Mary D Kingcome)                                                   |
|Williams, Mr. Charles Eugene                                    

In [10]:
# 4) Apenas as pessoas que não sobreviveram e eram do sexo masculino
# Desenvolva sua resposta aqui:

df.where("Survived <> 1 and Sex == 'male' ")\
  .select('Name','Sex')\
  .show(df.count(),truncate = False)

+--------------------------------------------------+----+
|Name                                              |Sex |
+--------------------------------------------------+----+
|Braund, Mr. Owen Harris                           |male|
|Allen, Mr. William Henry                          |male|
|Moran, Mr. James                                  |male|
|McCarthy, Mr. Timothy J                           |male|
|Palsson, Master. Gosta Leonard                    |male|
|Saundercock, Mr. William Henry                    |male|
|Andersson, Mr. Anders Johan                       |male|
|Rice, Master. Eugene                              |male|
|Fynney, Mr. Joseph J                              |male|
|Emir, Mr. Farred Chehab                           |male|
|Fortune, Mr. Charles Alexander                    |male|
|Todoroff, Mr. Lalio                               |male|
|Uruchurtu, Don. Manuel E                          |male|
|Wheadon, Mr. Edward H                             |male|
|Meyer, Mr. Ed

In [11]:
# 5) A média de tarifa paga para cada classe
# Desenvolva sua resposta aqui:

df.groupBy('Pclass').avg('Fare')\
  .withColumnRenamed('avg(Fare)','Average')\
  .withColumn('Average',round(('Average'),2))\
  .show(truncate = False)

+------+-------+
|Pclass|Average|
+------+-------+
|1     |84.15  |
|3     |13.68  |
|2     |20.66  |
+------+-------+



In [12]:
# 6) A média de tarifa paga para cada classe e sex
# Desenvolva sua resposta aqui:

df.groupBy('Pclass','Sex').avg('Fare')\
  .withColumnRenamed('avg(Fare)','Average')\
  .withColumn('Average',round(('Average'),2))\
  .show(truncate = False)

+------+------+-------+
|Pclass|Sex   |Average|
+------+------+-------+
|2     |female|21.97  |
|3     |male  |12.66  |
|1     |male  |67.23  |
|3     |female|16.12  |
|1     |female|106.13 |
|2     |male  |19.74  |
+------+------+-------+



In [13]:
# 7) Número de sobreviventes por classe e sexo
# Desenvolva sua resposta aqui:

df.where('Survived == 1')\
  .groupBy('Pclass','Sex').count()\
  .withColumnRenamed('count','Survivors')\
  .show(truncate = False)

+------+------+---------+
|Pclass|Sex   |Survivors|
+------+------+---------+
|2     |female|70       |
|3     |male  |47       |
|1     |male  |45       |
|3     |female|72       |
|1     |female|91       |
|2     |male  |17       |
+------+------+---------+



In [14]:
# 8) Número de sobreviventes por sexo e categorias de idade 
#    (as categorias de idade devem ser construídas como: abaixo de 18 = criança, 
#    entre 18 e 40 = jovem adulto, maior de 40 = adulto maduro).
# Desenvolva sua resposta aqui:

df.where('Survived == 1')\
                  .withColumn('Age Categories', when(df.Age < 18,'criança')\
                  .when((df.Age > 18) & (df.Age < 40),'jovem adulto')\
                  .when(df.Age > 40, 'adulto maduro')\
                  .when(df.Age.isNull(), df.Age))\
       .groupBy('Sex','Age Categories').count()\
       .withColumnRenamed('count','Survivors')\
       .na.drop()\
       .show(truncate = False)

+------+--------------+---------+
|Sex   |Age Categories|Survivors|
+------+--------------+---------+
|female|adulto maduro |37       |
|male  |criança       |23       |
|female|jovem adulto  |109      |
|male  |jovem adulto  |50       |
|male  |adulto maduro |18       |
|female|criança       |38       |
+------+--------------+---------+



# **03 - Escreve dados**

In [15]:
# Faça uma seleção apenas das pessoas que sobreviveram e 
# escreva os dados em um arquivo parquet particionado por sexo e classe.

In [35]:
df_parquet = df.where("Survived == 1")\
  .select('Name','Pclass','Sex')

In [ ]:
df_parquet.write.partitionBy('Pclass','Sex').parquet("/content/sample_data/writing_parquet/survivors.parquet")

In [47]:
spark.read.parquet('/content/sample_data/writing_parquet/').show(truncate = False)

+---------------------------------------------------+------+------+
|Name                                               |Pclass|Sex   |
+---------------------------------------------------+------+------+
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)|1     |female|
|Futrelle, Mrs. Jacques Heath (Lily May Peel)       |1     |female|
|Bonnell, Miss. Elizabeth                           |1     |female|
|Spencer, Mrs. William Augustus (Marie Eugenie)     |1     |female|
|Harper, Mrs. Henry Sleeper (Myna Haxtun)           |1     |female|
|Icard, Miss. Amelie                                |1     |female|
|Fortune, Miss. Mabel Helen                         |1     |female|
|Newsom, Miss. Helen Monypeny                       |1     |female|
|Pears, Mrs. Thomas (Edith Wearne)                  |1     |female|
|Chibnall, Mrs. (Edith Martha Bowerman)             |1     |female|
|Brown, Mrs. James Joseph (Margaret Tobin)          |1     |female|
|Lurette, Miss. Elise                           